<a href="https://colab.research.google.com/github/Neafiol/Tinkoff/blob/master/dssm/dssm_dz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seminar: simple question answering
![img](https://recruitlook.com/wp-content/uploads/2015/01/questionanswer3.jpg)

Today we're going to build a retrieval-based question answering model with metric learning models.

_this seminar is based on original notebook by [Oleg Vasilev](https://github.com/Omrigan/)_



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
%matplotlib inline

In [2]:
!wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall18/week11_dssm/utils.py

--2019-05-04 18:33:41--  https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall18/week11_dssm/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9814 (9.6K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   9.58K  --.-KB/s    in 0s      

2019-05-04 18:33:47 (107 MB/s) - ‘utils.py’ saved [9814/9814]



In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Dataset

Today's data is Stanford Question Answering Dataset (SQuAD). Given a paragraph of text and a question, our model's task is to select a snippet that answers the question.

We are not going to solve the full task today. Instead, we'll train a model to __select the sentence containing answer__ among several options.

As usual, you are given an utility module with data reader and some helper functions

In [0]:
import utils
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad-v2.0.json 2> log
# backup download link: https://www.dropbox.com/s/q4fuihaerqr0itj/squad.tar.gz?dl=1
train, test = utils.build_dataset('./squad-v2.0.json', tokenized=True)

In [5]:
# the data comes pre-tokenized with this simple tokenizer:
utils.tokenize("I... I'm the monument to all your sins.")

"i ... i ' m the monument to all your sins ."

In [6]:
pid, question, options, correct_indices, wrong_indices = train.iloc[40]
print('QUESTION', question, '\n')
for i, cand in enumerate(options):
    print(['[ ]', '[v]'][i in correct_indices], cand)

QUESTION where did beyonce get her name from ? 

[ ] beyoncé giselle knowles was born in houston , texas , to celestine ann " tina " knowles ( née beyincé ), a hairdresser and salon owner , and mathew knowles , a xerox sales manager .
[v] beyoncé ' s name is a tribute to her mother ' s maiden name .
[ ] beyoncé ' s younger sister solange is also a singer and a former member of destiny ' s child .
[ ] mathew is african - american , while tina is of louisiana creole descent ( with african , native american , french , cajun , and distant irish and spanish ancestry ).
[ ] through her mother , beyoncé is a descendant of acadian leader joseph broussard .
[ ] she was raised in a methodist household .


### Tokens & vocabularies

The procedure here is very similar to previous nlp weeks: preprocess text into tokens, create dictionaries, etc.

In [0]:
import re
mat=[]
for q in (train['question']):
  q=q.replace(',','')
  q=q.replace('.','')
  q=q.replace('?','')
  q=q.replace('`','')

  for w in q.split(' '):
    mat.append(w)

for o in (train["options"]):
  for q in o:
    q=q.replace(',','')
    q=q.replace('.','')
    q=q.replace('?','')
    q=q.replace('`','')

    for w in q.split(' '):
      mat.append(w)


In [8]:
len(mat)

9042915

In [0]:
from tqdm import tqdm, trange
from collections import Counter, defaultdict

#Dictionary of {token : count}
token_counts = Counter(mat)

# compute counts for each token; use token_counts;
# count BOTH in train['question'] and in train['options']


In [10]:
token_counts['me']

575

In [11]:
print("Total tokens:", sum(token_counts.values()))
print("Most common:", token_counts.most_common(5))
assert 9000000 < sum(token_counts.values()) < 9100000, "are you sure you counted all unique tokens in questions and options?"

Total tokens: 9042915
Most common: [('', 802262), ('the', 597790), ('of', 300056), ('and', 231619), ('in', 215312)]


We shall only keep tokens that are present at least 4 times

In [0]:
# token_counts=sorted(token_counts, key=lambda k: token_counts[k])
# token_counts.reverse()
tokens=token_counts

In [13]:
MIN_COUNT = 5

tokens = [c for c in token_counts if token_counts[c] >  MIN_COUNT] 
tokens = ["_PAD_", "_UNK_"] + tokens
print("Tokens left:", len(tokens))

Tokens left: 40801


In [0]:
# a dictionary from token to it's index in tokens
token_to_id={}
for i in range(len(tokens)) :
  token_to_id[tokens[i]] = i 

In [15]:
token_to_id["the"]

37

In [0]:
assert token_to_id['me'] != token_to_id['woods']
assert token_to_id[tokens[42]]==42
assert len(token_to_id)==len(tokens)

In [0]:
PAD_ix = token_to_id["_PAD_"]
UNK_ix = token_to_id['_UNK_']

#good old as_matrix for the third time
def as_matrix(sequences, max_len=None):
    if isinstance(sequences[0], (str, bytes)):
        sequences = [utils.tokenize(s).split() for s in sequences]
        
    max_len = max_len or max(map(len,sequences))
    
    matrix = np.zeros((len(sequences), max_len), dtype='int32') + PAD_ix
    for i, seq in enumerate(sequences):
        row_ix = [token_to_id.get(word, UNK_ix) for word in seq[:max_len]]
        matrix[i, :len(row_ix)] = row_ix
    
    return matrix

In [18]:
test = as_matrix(["Definitely, thOsE tokens areN'T LowerCASE!!", "I'm the monument to all your sins."])
print(test)
assert test.shape[0]==2
print("Correct!")

[[11174     1  1994  7720  7406    19   972 12207     1     0]
 [  196    19  2521    37  6800    49   647  6234 24359     1]]
Correct!


### Data sampler

Our model trains on triplets: $<query, answer^+, answer^->$

For your convenience, we've implemented a function that samples such triplets from data

In [0]:
import random
import torch
lines_to_tensor = lambda lines, max_len=None: torch.tensor(
    as_matrix(lines, max_len=max_len), dtype=torch.int64)

def iterate_minibatches(data, batch_size, shuffle=True, cycle=False):
    """
    Generates minibatches of triples: {questions, correct answers, wrong answers}
    If there are several wrong (or correct) answers, picks one at random.
    """
    indices = np.arange(len(data))
    while True:
        if shuffle:
            indices = np.random.permutation(indices)
        for batch_start in range(0, len(indices), batch_size):
            batch_indices = indices[batch_start: batch_start + batch_size]
            batch = data.iloc[batch_indices]
            questions = batch['question'].values
            correct_answers = np.array([
                row['options'][random.choice(row['correct_indices'])]
                for i, row in batch.iterrows()
            ])
            wrong_answers = np.array([
                row['options'][random.choice(row['wrong_indices'])]
                for i, row in batch.iterrows()
            ])

            yield {
                'questions' : lines_to_tensor(questions),
                'correct_answers': lines_to_tensor(correct_answers),
                'wrong_answers': lines_to_tensor(wrong_answers),
            }
        if not cycle:
            break

In [52]:
train

,paragraph_id,question,options,correct_indices,wrong_indices
0,0,when did beyonce start becoming popular ?,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[1],"[0, 2, 3]"
1,0,what areas did beyonce compete in when she was...,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[1],"[0, 2, 3]"
2,0,when did beyonce leave destiny ' s child and b...,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[3],"[0, 1, 2]"
3,0,in what city and state did beyonce grow up ?,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[1],"[0, 2, 3]"
4,0,in which decade did beyonce become famous ?,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[1],"[0, 2, 3]"
5,0,in what r & b group was she the lead singer ?,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[1],"[0, 2, 3]"
6,0,what album made her a worldwide known artist ?,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[3],"[0, 1, 2]"
7,0,who managed the destiny ' s child group ?,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[2],"[0, 1, 3]"
8,0,when did beyoncé rise to fame ?,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[1],"[0, 2, 3]"
9,0,what role did beyoncé have in destiny ' s child ?,[beyoncé giselle knowles - carter (/ biːˈjɒnse...,[1],"[0, 2, 3]"


In [48]:
dummy_batch = next(iterate_minibatches(train.sample(10), 2))
print(dummy_batch["questions"])
print(dummy_batch["correct_answers"])
print(dummy_batch["wrong_answers"])

tensor([[   2,    3,   37,  336,    1,   20,    1, 9675,  477, 1152,   12, 4091,
          353,   98,    1],
        [   4,  493,   49,   30,   44,   64,   39,   66,   37,  149,    1,    0,
            0,    0,    0]])
tensor([[   37,   336,     1,    20,     1,  9675,   477,  1152,    12,  4091,
           353,  1906,    12,   996,   381,     1,   695,    49,    37,   336,
             1,    20,     1,   455,  2721,    66,  3356,   817,  4245,     1,
          4246,    22,    37,  2203,  5278,  8014,    49,    52,  1906,   446,
            37,   620,   115,     1],
        [   13,    14,   394,    64,    39,    66,    37,   149,   584,    37,
           251,    14,   155,   329, 25970,    64,   229,  8684,  4584,    39,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]])
tensor([[   37,   787,   788,  2203,  5278,  2989,  6285,  8620,    37, 

### Building the model (3 points)

Our goal for today is to build a model that measures similarity between question and answer. In particular, it maps both question and answer into fixed-size vectors such that:

Our model is a pair of $V_q(q)$ and $V_a(a)$ - networks that turn phrases into vectors. 

__Objective:__ Question vector $V_q(q)$ should be __closer__ to correct answer vectors $V_a(a^+)$ than to incorrect ones $V_a(a^-)$ .

Both vectorizers can be anything you wish. For starters, let's use a convolutional network with global pooling and a couple of dense layers on top.

It is perfectly legal to share some layers between vectorizers, but make sure they are at least a little different.

In [0]:
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class GlobalMaxPooling(nn.Module):
    def __init__(self, dim=-1):
        super(self.__class__, self).__init__()
        self.dim = dim
        
    def forward(self, x):
        return x.max(dim=self.dim)[0]

In [0]:
# we might as well create a global embedding layer here

GLOBAL_EMB = nn.Embedding(len(tokens), 64, padding_idx=PAD_ix)

In [0]:
class QuestionVectorizer(nn.Module):

    def __init__(self, n_tokens=len(tokens), out_size=64, use_global_emb=True):
        """ 
        A simple sequential encoder for questions.
        Use any combination of layers you want to encode a variable-length input 
        to a fixed-size output vector
        
        If use_global_emb is True, use GLOBAL_EMB as your embedding layer
        """
        super(self.__class__, self).__init__()
        
        if use_global_emb:
            self.emb = GLOBAL_EMB
        else:
            self.emb = nn.Embedding(n_tokens, 64, padding_idx=PAD_ix)
            
        
        self.rnn = nn.RNN(64, out_size, num_layers =1 , dropout=0.05)
        
    def forward(self, text_ix):
        """
        :param text_ix: int64 Variable of shape [batch_size, max_len]
        :returns: float32 Variable of shape [batch_size, out_size]
        """
#         self.inp = nn.Linear(1, hidden_size)
       
        e = self.emb(text_ix)
        e = torch.transpose(e,1,0)
        
        h_0 = torch.zeros(e.shape)
      
        output, hn = self.rnn(e)
#         self.out = nn.Linear(hidden_size, 1)

        return output[-1]

In [0]:
class AnswerVectorizer(nn.Module):
    def __init__(self, n_tokens=len(tokens), out_size=64, use_global_emb=True):
        """ 
        A simple sequential encoder for questions.
        Use any combination of layers you want to encode a variable-length input 
        to a fixed-size output vector
        
        If use_global_emb is True, use GLOBAL_EMB as your embedding layer
        """
        super(self.__class__, self).__init__()
        
        if use_global_emb:
            self.emb = GLOBAL_EMB
        else:
            self.emb = nn.Embedding(n_tokens, 64, padding_idx=PAD_ix)
            
        
        self.rnn = nn.RNN(64, out_size, num_layers =1 , dropout=0.05)
        
    def forward(self, text_ix):
        """
        :param text_ix: int64 Variable of shape [batch_size, max_len]
        :returns: float32 Variable of shape [batch_size, out_size]
        """
#         self.inp = nn.Linear(1, hidden_size)
       
        e = self.emb(text_ix)
        e = torch.transpose(e,1,0)
        
        h_0 = torch.zeros(e.shape)
      
        output, hn = self.rnn(e)
#         self.out = nn.Linear(hidden_size, 1)

        return output[-1]

In [25]:
q=AnswerVectorizer(out_size=100)
q(torch.LongTensor(test)).shape

e:  torch.Size([10, 2, 64])


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


torch.Size([2, 100])

In [27]:
for vectorizer in [QuestionVectorizer(out_size=100), AnswerVectorizer(out_size=100)]:
  
    print("Testing %s ..." % vectorizer.__class__.__name__)
    dummy_x = torch.LongTensor(test)
    dummy_v = vectorizer(dummy_x)
    assert tuple(dummy_v.shape) == (dummy_x.shape[0], 100)

    del vectorizer
    print("Seems fine")

Testing QuestionVectorizer ...
e:  torch.Size([10, 2, 64])
Seems fine
Testing AnswerVectorizer ...
e:  torch.Size([10, 2, 64])
Seems fine


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [65]:
dummy_v.shape

torch.Size([2, 100])

### Training: loss function (3 points)
We want our vectorizers to put correct answers closer to question vectors and incorrect answers farther away from them. One way to express this is to use is Pairwise Hinge Loss _(aka Triplet Loss)_. 

$$ L = \frac 1N \underset {q, a^+, a^-} \sum max(0, \space \delta - sim[V_q(q), V_a(a^+)] + sim[V_q(q), V_a(a^-)] )$$

, where
* sim[a, b] is some similarity function: dot product, cosine or negative distance
* δ - loss hyperparameter, e.g. δ=1.0. If sim[a, b] is linear in b, all δ > 0 are equivalent.


This reads as __Correct answers must be closer than the wrong ones by at least δ.__

![img](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/margin.png)
<center>_image: question vector is green, correct answers are blue, incorrect answers are red_</center>


Note: in effect, we train a Deep Semantic Similarity Model [DSSM](https://www.microsoft.com/en-us/research/project/dssm/). 

In [0]:
def sim(a,b):
  return torch.dot(a.view(-1),b.view(-1))

def compute_loss(anchors, positives, negatives, delta=1):
    """ 
    Compute the triplet loss:
    
    max(0, delta + sim(anchors, negatives) - sim(anchors, positives))
    
    where sim is a dot-product between vectorized inputs
    
    """
#     loss = torch.max( delta + sim(anchors, negatives) - sim(anchors, positives) ,0)
    loss = delta + sim(anchors, negatives) - sim(anchors, positives)

    
    return loss

In [0]:
def compute_recall(anchors, positives, negatives, delta=1):
    """
    Compute the probability (ratio) at which sim(anchors, negatives) is greater than sim(anchors, positives)
    """
    ratio = sim(anchors, negatives)/sim(anchors, positives) 
    return ratio

In [106]:
_dummy_anchors=question_vectorizer(dummy_batch["questions"])
_dummy_positives = answer_vectorizer(dummy_batch["correct_answers"])
_dummy_negatives = answer_vectorizer(dummy_batch["wrong_answers"])
print( compute_loss(_dummy_anchors, _dummy_positives, _dummy_negatives) )
print( compute_recall(_dummy_anchors, _dummy_positives, _dummy_negatives) )

tensor(1.0219, grad_fn=<SubBackward0>)
tensor(1.0329, grad_fn=<DivBackward0>)


### Training loop (4 points)

For a difference, we'll ask __you__ to implement training loop this time.

Here's a sketch of one epoch:
1. iterate over __`batches_per_epoch`__ batches from __`train_data`__ with __`iterate_minibatches`__
    * Compute loss, backprop, optimize
    * Compute and accumulate recall
    
2. iterate over __`batches_per_epoch`__ batches from __`val_data`__
    * Compute and accumulate recall
    
3. print stuff :)


In [0]:
num_epochs = 100
max_len = 100
batch_size = 32
batches_per_epoch = 100

In [101]:
from itertools import chain

question_vectorizer = QuestionVectorizer(out_size=max_len)
answer_vectorizer = AnswerVectorizer(out_size=max_len)

optimizer = torch.optim.Adam(chain(question_vectorizer.parameters(),
                             answer_vectorizer.parameters()))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
for e in range(batches_per_epoch):
  for data in iterate_minibatches(train, batch_size):
    
    _dummy_anchors=question_vectorizer(dummy_batch["questions"])
    _dummy_positives = answer_vectorizer(dummy_batch["correct_answers"])
    _dummy_negatives = answer_vectorizer(dummy_batch["wrong_answers"])

  #   print( compute_loss(_dummy_anchors, _dummy_positives, _dummy_negatives) )
    

    loss = compute_loss(_dummy_anchors, _dummy_positives, _dummy_negatives)

    question_vectorizer.zero_grad()
    answer_vectorizer.zero_grad()

    loss.backward()
    optimizer.step()

  print( compute_recall(_dummy_anchors, _dummy_positives, _dummy_negatives) ) 

tensor(-0.7308, grad_fn=<DivBackward0>)
tensor(-1.2440, grad_fn=<DivBackward0>)
tensor(-1.2359, grad_fn=<DivBackward0>)


### Evaluation

Let's see how our model performs on actual question answering. You will score answer candidates with your model and select the most appropriate one.

__Your goal__ is to obtain accuracy of at least above 50%. Beating 65% in this notebook yields bonus points :)

In [0]:
# optional: prepare some functions here
# <...>

def select_best_answer(question, possible_answers):
  
    """
    Predicts which answer best fits the question
    :param question: a single string containing a question
    :param possible_answers: a list of strings containing possible answers
    :returns: integer - the index of best answer in possible_answer
    """
  
    q=question_vectorizer(question)
    nans=0
    now = 1e9
    
    for i , a in enumirate(possible_answers):
      ans=answer_vectorizer(a)
      if(sim(q, ans)< now):
        now = sim(q, ans)
        nans=i
        
    return possible_answers[nans]
    

    

In [0]:
predicted_answers = [
    select_best_answer(question, possible_answers)
    for i, (question, possible_answers) in tqdm(test[['question', 'options']].iterrows(), total=len(test))
]

accuracy = np.mean([
    answer in correct_ix
    for answer, correct_ix in zip(predicted_answers, test['correct_indices'].values)
])
print("Accuracy: %0.5f" % accuracy)
assert accuracy > 0.65, "we need more accuracy!"
print("Great job!")

In [0]:
def draw_results(question, possible_answers, predicted_index, correct_indices):
    print("Q:", question, end='\n\n')
    for i, answer in enumerate(possible_answers):
        print("#%i: %s %s" % (i, '[*]' if i == predicted_index else '[ ]', answer))
    
    print("\nVerdict:", "CORRECT" if predicted_index in correct_indices else "INCORRECT", 
          "(ref: %s)" % correct_indices, end='\n' * 3)

In [0]:
for i in [1, 100, 1000, 2000, 3000, 4000, 5000]:
    draw_results(test.iloc[i].question, test.iloc[i].options,
                 predicted_answers[i], test.iloc[i].correct_indices)

In [0]:
question = "What is my name?" # your question here!
possible_answers = [
    <...> 
    # ^- your options. 
]
predicted answer = select_best_answer(question, possible_answers)

draw_results(question, possible_answers,
             predicted_answer, [0])

### Bonus tasks

There are many ways to improve our question answering model. Here's a bunch of things you can do to increase your understanding and get bonus points.


### 0. Fine-tuning (3+ pts)
This time our dataset is fairly small. We can improve the training procedure by starting with a pre-trained model.
* The simplest option is to use pre-trained embeddings. See previous weeks for that.
* A harder (but better) alternative is to use a pre-trained sentence encoder. Consider [InferSent](https://github.com/facebookresearch/InferSent), Universal Sentence Encoder or ELMO.


### 1.  Hard Negatives (3+ pts)

Not all wrong answers are equally wrong. As the training progresses, _most negative examples $a^-$ will be to easy._ So easy in fact, that loss function and gradients on such negatives is exactly __0.0__. To improve training efficiency, one can __mine hard negative samples__.

Given a list of answers,
* __Hard negative__ is the wrong answer with highest similarity with question,

$$a^-_{hard} = \underset {a^-} {argmax} \space sim[V_q(q), V_a(a^-)]$$

* __Semi-hard negative__ is the one with highest similarity _among wrong answers that are farther than positive one. This option is more useful if some wrong answers may actually be mislabelled correct answers.

* One can also __sample__ negatives proportionally to $$P(a^-_i) \sim e ^ {sim[V_q(q), V_a(a^-_i)]}$$


The task is to implement at least __hard negative__ sampling and apply it for model training.


### 2. Bring Your Own Model (3+ pts)
In addition to Universal Sentence Encoder, one can also train a new model.
* You name it: convolutions, RNN, self-attention
* Use pre-trained ELMO or FastText embeddings
* Monitor overfitting and use dropout / word dropout to improve performance

__Note:__ if you use ELMO please note that it requires tokenized text while USE can deal with raw strings. You can tokenize data manually or use tokenized=True when reading dataset.


* hard negatives (strategies: hardest, hardest farter than current, randomized)
* train model on the full dataset to see if it can mine answers to new questions over the entire wikipedia. Use approximate nearest neighbor search for fast lookup.


### 3. Search engine (3+ pts)

Our basic model only selects answers from 2-5 available sentences in paragraph. You can extend it to search over __the whole dataset__. All sentences in all other paragraphs are viable answers.

The goal is to train such a model and use it to __quickly find top-10 answers from the whole set__.

* You can ask such model a question of your own making - to see which answers it can find among the entire training dataset or even the entire wikipedia.
* Searching for top-K neighbors is easier if you use specialized methods: [KD-Tree](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html) or [HNSW](https://github.com/nmslib/hnswlib). 
* This task is much easier to train if you use hard or semi-hard negatives. You can even find hard negatives for one question from correct answers to other questions in batch - do so in-graph for maximum efficiency. See [1.] for more details.
